## Financial Market Analytics Project

In [ ]:
# All imports
import pandas as pd
import numpy as np
from scipy.stats import skew
from datetime import datetime, timedelta
import plotly.express as px

In [7]:
# Read the data, skipping the first row and using the second row as headers
# Read the data, skipping the first row and using the second row as headers
df_sp_m_new = pd.read_excel("../Datasets/SPX500.xlsm", sheet_name="Price", engine="openpyxl", skiprows=[0], header=0)
df_stoxx_m_new = pd.read_excel("../Datasets/Stoxx600.xlsm", sheet_name="Price", engine="openpyxl", skiprows=[0], header=0)

df_sp_d_new = pd.read_excel("../Datasets/SPX500.xlsm", sheet_name="Price daily", engine="openpyxl", skiprows=[0], header=0)
df_stoxx_d_new = pd.read_excel("../Datasets/Stoxx600.xlsm", sheet_name="Price daily", engine="openpyxl", skiprows=[0], header=0)


# Flatten the MultiIndex columns created by read_excel with header=0 and skiprows=[0]
df_sp_m_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sp_m_new.columns]
df_stoxx_m_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_stoxx_m_new.columns]
df_sp_d_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sp_d_new.columns]
df_stoxx_d_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_stoxx_d_new.columns]


print("Nuovo df_sp_m head:")
display(df_sp_m_new.head())

KeyboardInterrupt: 

### Skewness
In this step, we filter the dataset to include only the last six months of data, then calculate the percentage of missing values for each stock. We retain only those stocks with no more than 30% missing data (excluding the 'Dates' column). Next, we compute the skewness of returns for the selected stocks, sort them by skewness, and finally select the top 20% with the highest skewness values.

In [ ]:
# Convert all columns except 'Dates' to numeric, coercing errors
for column in df_sp_d_new.columns:
    if column != 'Dates':
        df_sp_d_new[column] = pd.to_numeric(df_sp_d_new[column], errors='coerce')

# Convert 'Dates' column to datetime objects
df_sp_d_new['Dates'] = pd.to_datetime(df_sp_d_new['Dates'])

# Filter for the last 6 months
six_months_ago = df_sp_d_new['Dates'].max() - pd.DateOffset(months=6)
df_sp_d_last_6m_new = df_sp_d_new[df_sp_d_new['Dates'] >= six_months_ago]

# Calculate missing values percentage for each column
missing_percentage_new = df_sp_d_last_6m_new.isnull().mean() * 100

# Select columns with less than or equal to 30% missing values, excluding 'Dates'
cols_to_analyze_new = missing_percentage_new[missing_percentage_new <= 30].index.tolist()
if 'Dates' in cols_to_analyze_new:
    cols_to_analyze_new.remove('Dates')

# Calculate skewness for the selected columns
skewness_values_new = df_sp_d_last_6m_new[cols_to_analyze_new].skew()

# Sort skewness values and select the top 20%
top_20_percent_count_new = int(len(skewness_values_new) * 0.2)
top_skewness_new = skewness_values_new.sort_values(ascending=False).head(top_20_percent_count_new)

print("Top 20% titles with highest skewness in the last 6 months:")
print(top_skewness_new)

Top 20% titles with highest skewness in the last 6 months:
Keurig Dr Pepper Inc          1.377693
Edwards Lifesciences Corp     1.291127
Graham Holdings Co            1.289603
Big Lots Inc                  1.251414
McDonald's Corp               1.127863
                                ...   
Wachovia Corp/Old             0.000000
Lexmark International Inc     0.000000
ConocoPhillips Holding Co     0.000000
Burlington Resources Inc      0.000000
CommScope Connectivity LLC    0.000000
Length: 270, dtype: float64


Scatterplot for the skewness for the entire S&P500, and scatterplot of the top 20% titles of S&P500 with highest skewness, in the last 6 months.

In [ ]:
fig = px.scatter(
    x=skewness_values_new.index,
    y=skewness_values_new.values,
    labels={'x': 'Stock Ticker', 'y': 'Skewness'},
    title='Skewness of S&P 500 Stocks (Last 6 Months)'
)

fig.show()

fig2 = px.scatter(
    x=top_skewness_new.index,
    y=top_skewness_new.values,
    labels={'x': 'Stock Ticker', 'y': 'Skewness'},
    title='Topo 20% Skewness of S&P 500 Stocks (Last 6 Months)'
)

fig2.show()

### Comparison with the Markovitz optimal portfolio
Comparing a Markowitz mean-variance optimal portfolio with our momentum + skewness strategy is a great way to validate whether this factor-based method adds value. The S&P500 dataset will be used.